In [5]:
import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()

dir='/Users/xinwang/ai/dataset/kaggle/GStore/'
train_file = 'train.csv'
test_file = 'test.csv'

train = pd.read_csv(dir + train_file, low_memory=False)
test = pd.read_csv(dir + test_file, low_memory=False)


cate_features = []
numeric_features = []

train['fullVisitorId'] = train['fullVisitorId'].astype(str)
test['fullVisitorId'] = test['fullVisitorId'].astype(str)

train['channelGrouping'] = label.fit_transform(train['channelGrouping'])
test['channelGrouping'] = label.fit_transform(test['channelGrouping'])

cate_features.append('channelGrouping')

def label_transform(df, col_list):
    for col in col_list:
        df[col] = label.fit_transform(df[col])

    return df

####################################
date_features = ['date','year','month','day','week','weekofyear','dayofweek','quarter','month_start','month_end']

def process_date(df):
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d',errors='ignore')
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day

    df['week'] = df['date'].dt.week
    df['weekofyear'] = df['date'].dt.weekofyear
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month_start'] = df['date'].dt.is_month_start
    df['month_end'] = df['date'].dt.is_month_end

    df = label_transform(df, date_features)
    
    return df

train = process_date(train)
test = process_date(test)
cate_features += date_features
print('process_date done')

################device####################
device_features = ['browser','operatingSystem','isMobile','deviceCategory']

def process_device(df):
    df['browser'] = df['device'].apply(lambda x: json.loads(x)['browser'])
    df['operatingSystem'] = df['device'].apply(lambda x: json.loads(x)['operatingSystem'])
    df['isMobile'] = df['device'].apply(lambda x: json.loads(x)['isMobile'])
    df['deviceCategory'] = df['device'].apply(lambda x: json.loads(x)['deviceCategory'])

    df = label_transform(df, device_features)

    return df

train = process_device(train)
test = process_device(test)
cate_features += device_features
print('process_device done')


###############geoNetwork#####################
geo_features = ['continent','subContinent','country','region','metro','city','networkDomain']

def process_geo(df):
    df['continent'] = df['geoNetwork'].apply(lambda x: json.loads(x)['continent'])
    df['subContinent'] = df['geoNetwork'].apply(lambda x: json.loads(x)['subContinent'])
    df['country'] = df['geoNetwork'].apply(lambda x: json.loads(x)['country'])
    df['region'] = df['geoNetwork'].apply(lambda x: json.loads(x)['region'])
    df['metro'] = df['geoNetwork'].apply(lambda x: json.loads(x)['metro'])
    df['city'] = df['geoNetwork'].apply(lambda x: json.loads(x)['city'])
    df['networkDomain'] = df['geoNetwork'].apply(lambda x: json.loads(x)['networkDomain'])

    df = label_transform(df, geo_features)

    return df

train = process_geo(train)
test = process_geo(test)
cate_features += geo_features
print('process_geo done')


################totals####################
view_features = ['hits','pageviews','newVisits','bounces','visitNumber']

def process_totals(df):
    df['hits'] = df['totals'].apply(lambda x: json.loads(x)['hits']).astype(int)
    df['pageviews'] = df['totals'].apply(lambda x: json.loads(x)['pageviews'] if x.find('pageviews')>=0 else 0).astype(int)
    df['bounces'] = df['totals'].apply(lambda x: json.loads(x)['bounces'] if x.find('bounces')>=0 else 0).astype(int)
    df['newVisits'] = df['totals'].apply(lambda x: json.loads(x)['newVisits'] if x.find('newVisits')>=0 else 0).astype(int)
    df['transactionRevenue'] = df['totals'].apply(lambda x: json.loads(x)['transactionRevenue'] if x.find('transactionRevenue')>=0 else 0).astype(int)

    return df

train = process_totals(train)
test = process_totals(test)
numeric_features += view_features
print('process_totals done')


################totals####################
last_time_features = ['last_seconds','last_minutes']

def process_last_time(df):
    df['last_seconds'] = df['visitStartTime']-df['visitId']
    df['last_minutes'] = (df['visitStartTime']-df['visitId'])/60
    df['last_minutes'] = df['last_minutes'].astype(np.int64)

    return df

train = process_last_time(train)
test = process_last_time(test)
numeric_features += last_time_features
print('process_last_time done')


################Multip transaction####################
def process_transaction_count(df):
    transaction_df = train[['fullVisitorId','transactionRevenue']].groupby('fullVisitorId')
        
    transac_count_df = transaction_df.count()
    transac_count_df['fullVisitorId'] = transac_count_df.index
    transac_count_df.rename(columns={
        'transactionRevenue':'transaction_count'
    }, inplace=True)
    transac_count_df['transaction_count']=transac_count_df['transaction_count'].astype(int)

    df = pd.merge(df, transac_count_df, on='fullVisitorId')

    return df

train = process_transaction_count(train)
test = process_transaction_count(test)
numeric_features.append('transaction_count')
print('process_transaction_count done')


def process_buy_times(df):
    transaction_df = df[['fullVisitorId','transactionRevenue']].groupby('fullVisitorId')

    def count_buy_times(x):
        buy_times = sum(x['transactionRevenue']>0)

        return buy_times

    buy_times_group = transaction_df.apply(count_buy_times)
    buy_times_df = pd.DataFrame({
        'fullVisitorId':buy_times_group.index,
        'buy_times':buy_times_group.values
    })

    df = pd.merge(df, buy_times_df, on='fullVisitorId')

    return df

train = process_buy_times(train)
test = process_buy_times(test)
numeric_features.append('buy_times')
print('process_buy_times done')


def parse_adwordsClickInfo_field(x, field):
    jo = json.loads(x)
    
    if x.find('adwordsClickInfo')>=0:
        adwordsClickInfo = jo['adwordsClickInfo']
        
        if str(adwordsClickInfo).find(field)>=0:
            return adwordsClickInfo[field]

    return 0

def parse_adwordsClickInfo_page(x):
    return parse_adwordsClickInfo_field(x, 'page')

def parse_adwordsClickInfo_slot(x):
    return parse_adwordsClickInfo_field(x, 'slot')

def parse_adwordsClickInfo_gclId(x):
    return parse_adwordsClickInfo_field(x, 'gclId')

def parse_adwordsClickInfo_adNetworkType(x):
    return parse_adwordsClickInfo_field(x, 'adNetworkType')

def parse_adwordsClickInfo_isVideoAd(x):
    return parse_adwordsClickInfo_field(x, 'isVideoAd')

traffic_features = ['campaign','source','medium','keyword','adwordsClickInfo_gclId_prefix','adwordsClickInfo_slot',
                    'adwordsClickInfo_gclId','adwordsClickInfo_adNetworkType']

def process_traffic(df):
    df['campaign'] = df['trafficSource'].apply(lambda x: json.loads(x)['campaign']).astype(str)
    # need to merge nearly same record
    df['source'] = df['trafficSource'].apply(lambda x: json.loads(x)['source']).astype(str)
    df['medium'] = df['trafficSource'].apply(lambda x: json.loads(x)['medium']).astype(str)
    # need to merge some keywords
    df['keyword'] = df['trafficSource'].apply(lambda x: json.loads(x)['keyword'] if x.find('keyword')>=0 else 0).astype(str)

    df['adwordsClickInfo_page'] = df['trafficSource'].apply(parse_adwordsClickInfo_page).astype(int)
    df['adwordsClickInfo_slot'] = df['trafficSource'].apply(parse_adwordsClickInfo_slot).astype(str)
    df['adwordsClickInfo_gclId'] = df['trafficSource'].apply(parse_adwordsClickInfo_gclId).astype(str)
    df['adwordsClickInfo_gclId_prefix'] = df['adwordsClickInfo_gclId'].apply(lambda x: x.split('_')[0] if type(x)!=int and x.find('_')>=0 else 0).astype(str)
    df['adwordsClickInfo_adNetworkType'] = df['trafficSource'].apply(parse_adwordsClickInfo_adNetworkType).astype(str)

    df = label_transform(df, traffic_features)
    
    return df

train = process_traffic(train)
test = process_traffic(test)

cate_features += traffic_features
numeric_features.append('adwordsClickInfo_page')
print('process_traffic done')


###################################################### 
target = 'revenue'

def process_revenue(df):
    revenue_df = df[['fullVisitorId','transactionRevenue']].groupby('fullVisitorId').agg('sum')
    revenue_df['fullVisitorId'] = revenue_df.index
    revenue_df[target] = revenue_df['transactionRevenue'].apply(lambda x: np.log(x+1))

    revenue_df.drop('transactionRevenue', axis=1, inplace=True)
    
    df = pd.merge(df, revenue_df, on='fullVisitorId')

    return df


train = process_revenue(train)
test = process_revenue(test)
print('process_revenue done')

removed_columns = ['device','geoNetwork','socialEngagementType','totals','trafficSource','transactionRevenue']
train.drop(removed_columns, axis=1, inplace=True)


train[target].values[0:10]

process_date done
process_device done
process_geo done
process_totals done
process_last_time done


/Users/xinwang/ai/lib/python3.6/site-packages/ipykernel_launcher.py:148: FutureWarning: 'fullVisitorId' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
/Users/xinwang/ai/lib/python3.6/site-packages/ipykernel_launcher.py:149: FutureWarning: 'fullVisitorId' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version


process_transaction_count done
process_buy_times done
process_traffic done


/Users/xinwang/ai/lib/python3.6/site-packages/ipykernel_launcher.py:248: FutureWarning: 'fullVisitorId' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
/Users/xinwang/ai/lib/python3.6/site-packages/ipykernel_launcher.py:249: FutureWarning: 'fullVisitorId' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version


process_revenue done


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [6]:
# Parallet groupby works well
# Merge the same fullVisitorId records into one record

from joblib import Parallel, delayed
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
import time


def applyParallel(dfGrouped, func):
    with Pool(cpu_count()) as p:
        ret_list = p.map(func, [group for name, group in dfGrouped])

    df = pd.concat(ret_list, axis=1)
    df = df.T
    return df


def merge_group_func(df):
    def merge_one_column(x):
        new_merged_value=x.mode().values[0]
        return new_merged_value

    temp = df.apply(merge_one_column)

    return temp

def merge_duplicated_into_new_row(df):
    print('unique fullVisitorId size ',len(df['fullVisitorId'].unique()))

    d_rows = df[df['fullVisitorId'].duplicated(keep=False)]
    revisted_df = df.loc[d_rows.index]
    print('revisted_df.shape',revisted_df.shape)

    new_merged_df = applyParallel(revisted_df.groupby('fullVisitorId'),merge_group_func)
    print('new_merged_df.shape',new_merged_df.shape)

    df.drop(d_rows.index, inplace=True)
    
    df = pd.concat([df,new_merged_df], axis=0)
    del d_rows
    del new_merged_df
    
    print('df.shape',df.shape)

    return df
    

train = merge_duplicated_into_new_row(train)
test = merge_duplicated_into_new_row(test)
print('merge_duplicated_into_new_row done')

print('train.shape', train.shape)
train.head(10)

unique fullVisitorId size  714167
revisted_df.shape (282978, 45)
new_merged_df.shape (93492, 45)
df.shape (714167, 45)
unique fullVisitorId size  7679
revisted_df.shape (15844, 51)
new_merged_df.shape (3239, 51)
df.shape (7679, 51)
merge_duplicated_into_new_row done
train.shape (714167, 45)


,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,year,month,day,...,campaign,source,medium,keyword,adwordsClickInfo_page,adwordsClickInfo_slot,adwordsClickInfo_gclId,adwordsClickInfo_gclId_prefix,adwordsClickInfo_adNetworkType,revenue
0,4,32,1131660440785968503,1131660440785968503_1472830385,1472830385,1,1472830385,0,8,1,...,0,149,5,11,0,0,0,0,0,0
1,4,32,377306020877927890,377306020877927890_1472880147,1472880147,1,1472880147,0,8,1,...,0,149,5,11,0,0,0,0,0,0
2,4,32,3895546263509774583,3895546263509774583_1472865386,1472865386,1,1472865386,0,8,1,...,0,149,5,11,0,0,0,0,0,0
3,4,32,4763447161404445595,4763447161404445595_1472881213,1472881213,1,1472881213,0,8,1,...,0,149,5,1099,0,0,0,0,0,0
6,4,32,2938943183656635653,2938943183656635653_1472807194,1472807194,1,1472807194,0,8,1,...,0,149,5,11,0,0,0,0,0,0
7,4,32,1905672039242460897,1905672039242460897_1472817241,1472817241,1,1472817241,0,8,1,...,0,149,5,11,0,0,0,0,0,0
8,4,32,537222803633850821,537222803633850821_1472812602,1472812602,1,1472812602,0,8,1,...,0,149,5,11,0,0,0,0,0,0
9,4,32,4445454811831400414,4445454811831400414_1472805784,1472805784,1,1472805784,0,8,1,...,0,149,5,11,0,0,0,0,0,0
10,4,32,9499785259412240342,9499785259412240342_1472812272,1472812272,1,1472812272,0,8,1,...,0,149,5,11,0,0,0,0,0,0
11,4,32,0523069750702990437,0523069750702990437_1472834967,1472834967,1,1472834967,0,8,1,...,0,149,5,11,0,0,0,0,0,0


In [7]:
from keras.layers import Input, Dense, concatenate, Dropout, Embedding, Flatten
from keras.models import Model
from keras import optimizers

def get_model():
    input_cate = Input((len(cate_features),))
    input_numeric = Input((len(numeric_features),))
    
    x_cate = Embedding(100000, 10)(input_cate)
    x_cate = Flatten()(x_cate)
    x_cate = Dropout(0.2)(x_cate)
    x_cate = Dense(100, activation='relu')(x_cate)
    
    
    x_numeric = Dense(100, activation='relu')(input_numeric)
    x_numeric = Dropout(0.2)(x_numeric)
    
    x = concatenate([x_cate,x_numeric])
    
    x = Dense(100, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    output = Dense(1, kernel_initializer='normal')(x)
    
    model = Model(inputs=[input_cate,input_numeric], outputs=output)
    model.compile(optimizer=optimizers.Adam(), loss='mean_squared_error', metrics=['mse'])
    
    return model
    
    
model = get_model()
print(model.summary())
model.fit([train[cate_features], train[numeric_features]], train[target], validation_split=0.2, epochs=5, batch_size=100)
print('model.fit done')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 30, 10)       1000000     input_3[0][0]                    
__________________________________________________________________________________________________
flatten_2 (Flatten)             (None, 300)          0           embedding_2[0][0]                
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
dropout_4 

In [8]:
predict_test = model.predict([test[cate_features], test[numeric_features]])

test['PredictedLogRevenue'] = predict_test


print('predict done')

predict done


In [9]:
target_sum = test[['fullVisitorId','PredictedLogRevenue']].groupby('fullVisitorId', as_index=False).agg('sum')


target_sum.to_csv('GStore_keras_baseline.csv', index=False)
print(target_sum.shape)

(7679, 2)
